In [0]:
# import dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np

In [0]:
# Extracting all group urls from howStat.com
request = requests.get('http://www.howstat.com/cricket/Statistics/Players/PlayerMenu.asp')
response = request.text

soup = bs(response)

list_a = soup.find_all('a', class_= 'abcList')

list_of_all_group_link = []

for link in list_a:
    url = "http://www.howstat.com/cricket/Statistics/Players/" + link['href']
    list_of_all_group_link.append(url)

list_of_all_group_link

['http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=A',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=B',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=C',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=D',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=E',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=F',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=G',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=H',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=I',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=J',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=K',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=L',
 'http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=M',

In [0]:
# req = requests.get('http://www.howstat.com/cricket/Statistics/Players/PlayerList.asp?Group=A')
# res = req.text

# soup = bs(res)

# table = soup.find_all('table')[5]

# td = table.find_all('td', width = 125)

# td[0].text

# Getting the country names of all ODI Players

player_country = []

for each_alpha_url in list_of_all_group_link :
  each_alpha_req = requests.get(each_alpha_url)
  each_alpha_response = each_alpha_req.text
  
  soup = bs(each_alpha_response)
  
  table = soup.find_all('table')[5]
  td = table.find_all('td', width = 125)
  for each_td in td :
    txt = each_td.text
    txt = txt.replace('\r\n', '')
    txt = txt.replace(' ', '')
    player_country.append(txt)

In [0]:
player_country[:11]

['Pakistan',
 'Pakistan',
 'Pakistan',
 'Pakistan',
 'Oman',
 'Pakistan',
 'Pakistan',
 'Pakistan',
 'Nepal',
 'India',
 'Kenya']

In [0]:
# Getting the names and ID of all ODI Players

player_name = []
player_id = []

for each_alpha_url in list_of_all_group_link:
    each_alpha_req = requests.get(each_alpha_url)
    each_alpha_response = each_alpha_req.text
    
    each_alpha_soup = bs(each_alpha_response)
    
    raw_list_of_players = each_alpha_soup.find_all('a', class_ = 'LinkNormal')
    
    for player in raw_list_of_players:
        hreftxt = player['href']
        if hreftxt[:21] == 'PlayerOverviewSummary':
            name = player.text
            iD = hreftxt[-4:]
            player_name.append(name)
            player_id.append(iD)

In [0]:
# Result so far..
print(player_name[:6])
print(player_id[:6])
print(player_country[:6])
print(len(player_name))

['Aamer Hameed', 'Aamer Hanif', 'Aamer Malik', 'Aamer Yamin', 'Aamir Kaleem', 'Aamir Nazir']
['2601', '2602', '1068', '4480', '4465', '1259']
['Pakistan', 'Pakistan', 'Pakistan', 'Pakistan', 'Oman', 'Pakistan']
4478


In [0]:
for i,name in enumerate(player_name):
  name = name.replace(',','.')
  player_name[i] = name

In [0]:
# Storing the data into a csv file
data = {
    'Player Name' : player_name,
    'Player_ID' : player_id,
    'Player_country': player_country
}

df = pd.DataFrame(data)
print(df.head(10))

df.to_csv('NameId.csv')

    Player Name Player_ID Player_country
0  Aamer Hameed      2601       Pakistan
1   Aamer Hanif      2602       Pakistan
2   Aamer Malik      1068       Pakistan
3   Aamer Yamin      4480       Pakistan
4  Aamir Kaleem      4465           Oman
5   Aamir Nazir      1259       Pakistan
6  Aamir Sohail      1653       Pakistan
7   Aaqib Javed      0869       Pakistan
8  Aarif Sheikh      4808          Nepal
9    Aaron. V R      3916          India


In [0]:
# Now, the real task, Getting year wise runs !
all_player_runs = []
id_to_remove = []
count = 0
for player in player_id:
    player_url = 'http://www.howstat.com/cricket/Statistics/Players/PlayerYears_ODI.asp?PlayerID=' + player
    runs_yearwise_all = np.zeros(50)
    player_year = []
    
    req = requests.get(player_url)
    response = req.text
    player_soup = bs(response)
    
    h2 = player_soup.find('h2')
    if h2 != None:
        id_to_remove.append(player)
        continue
    
    player_stats = player_soup.find('div', id = "bat")
    player_year_temp = player_stats.find_all('a', class_='LinkNormal')
    
    for y in player_year_temp:
        y = y.text
        player_year.append(y)
    
    player_table = player_soup.find_all('table')[7]
    tds = player_table.find_all('td')
    tds = tds[13:]
    p_run = []
    
    for i, each_td in enumerate(tds):
        if (i + 1) % 13 == 9:
            run = each_td.text
            run.replace('\r\n', '')
            run.replace(' ', '')
            p_run.append(run)
    
    for (y, r) in zip(player_year, p_run):
        year_int = int(y)
        indx = year_int - 1970
        runs_yearwise_all[indx] = r
    count = count + 1
    print(count)
    all_player_runs.append(runs_yearwise_all)
print("Done !")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
print(id_to_remove)

['4465', '4868', '5173', '0902', '5105', '4870', '3694', '5096', '0001', '0005', '5128', '5137', '5138', '5129', '0007', '5177', '0008', '2128', '0010', '5127', '0011', '4867', '4497', '0819', '4734', '0123', '0672', '2976', '4104', '0036', '1962', '5010', '4446', '4880', '5097', '5035', '4440', '5169', '0030', '3025', '0033', '0034', '0035', '4998', '0040', '0039', '2117', '0048', '0049', '0050', '0051', '0052', '0053', '0054', '0055', '0056', '0057', '0061', '0062', '4887', '0066', '0067', '4468', '0504', '5113', '4501', '4860', '4355', '4862', '0070', '0072', '0073', '0074', '0075', '5139', '5043', '5135', '4471', '0820', '0926', '0082', '5015', '0083', '0084', '5013', '5187', '4120', '0085', '0086', '0087', '0270', '5146', '0089', '0090', '3776', '0091', '0093', '5009', '5080', '3760', '5114', '0097', '5107', '0098', '3137', '0099', '5134', '5141', '4616', '5005', '1974', '0103', '0104', '0105', '3948', '3709', '5002', '4910', '3942', '3982', '5122', '0927', '5101', '5132', '4866',

In [0]:
df2 = df
df2 = df.set_index('Player_ID')
df2.drop(id_to_remove, inplace = True)
df2.head()

,Player Name,Player_country
Player_ID,,
2601,Aamer Hameed,Pakistan
2602,Aamer Hanif,Pakistan
1068,Aamer Malik,Pakistan
4480,Aamer Yamin,Pakistan
1259,Aamir Nazir,Pakistan


In [0]:
df2.to_csv('NameIdFiltered.csv')

In [0]:
len(all_player_runs)

2547

In [0]:
headers = list(range(1970, 2020))
print(headers)

[1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


In [0]:
all_player_runs

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 17.,  0., 72.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,  31., 468.,  37.,   0.,
          0.,   0.,  20.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.]),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
  

In [0]:
dafr = pd.DataFrame(all_player_runs, columns = headers)
dafr.to_csv('Runs.csv')

In [0]:
df3 = df2.reset_index()
df3.head()

,Player_ID,Player Name,Player_country
0,2601,Aamer Hameed,Pakistan
1,2602,Aamer Hanif,Pakistan
2,1068,Aamer Malik,Pakistan
3,4480,Aamer Yamin,Pakistan
4,1259,Aamir Nazir,Pakistan


In [0]:
df4 = pd.merge(df3, dafr, right_index=True, left_index=True)
df4.head()

,Player_ID,Player Name,Player_country,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,2601,Aamer Hameed,Pakistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2602,Aamer Hanif,Pakistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1068,Aamer Malik,Pakistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.0,468.0,37.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4480,Aamer Yamin,Pakistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.0,0.0,0.0,32.0,0.0
4,1259,Aamir Nazir,Pakistan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
df4.to_csv('FinalDataSet_withoutTotal.csv')

In [0]:
df4.dtypes

Player_ID          object
Player Name        object
Player_country     object
1970              float64
1971              float64
1972              float64
1973              float64
1974              float64
1975              float64
1976              float64
1977              float64
1978              float64
1979              float64
1980              float64
1981              float64
1982              float64
1983              float64
1984              float64
1985              float64
1986              float64
1987              float64
1988              float64
1989              float64
1990              float64
1991              float64
1992              float64
1993              float64
1994              float64
1995              float64
1996              float64
1997              float64
1998              float64
1999              float64
2000              float64
2001              float64
2002              float64
2003              float64
2004              float64
2005        

In [0]:
df4['Total Runs'] = df4.iloc[:, 2:-1].sum(axis = 1)
df4.head()

In [0]:
df4.to_csv("FinalData_Avi_Kasliwal.csv")